# PYTHON LIBRARIES

LOADING PYTHON LIBRARIES

In [199]:
# dash Libraries
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

# plotly Libraries
import plotly.graph_objects as go
import dash_table

# pandas Libraries
import pandas as pd


# keras Library
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

# numpy Library
import numpy as np

DATA CLEANING AND SCALING

In [200]:
# dataset reading
df_nse = pd.read_csv("data/NSE-Tata.csv")
df = pd.read_csv("data/stock_data.csv")

In [201]:
df_nse

,Date,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
0,2018-10-08,208.00,222.25,206.85,216.00,215.15,4642146.0,10062.83
1,2018-10-05,217.00,218.60,205.90,210.25,209.20,3519515.0,7407.06
2,2018-10-04,223.50,227.80,216.15,217.25,218.20,1728786.0,3815.79
3,2018-10-03,230.00,237.50,225.75,226.45,227.60,1708590.0,3960.27
4,2018-10-01,234.55,234.60,221.05,230.30,230.90,1534749.0,3486.05
...,...,...,...,...,...,...,...,...
1230,2013-10-14,160.85,161.45,157.70,159.30,159.45,1281419.0,2039.09
1231,2013-10-11,161.15,163.45,159.00,159.80,160.05,1880046.0,3030.76
1232,2013-10-10,156.00,160.80,155.85,160.30,160.15,3124853.0,4978.80
1233,2013-10-09,155.70,158.20,154.15,155.30,155.55,2049580.0,3204.49


In [202]:
# index formatting
df_nse["Date"]=pd.to_datetime(df_nse.Date,format="%Y-%m-%d")
df_nse.index=df_nse['Date']

In [203]:
# data sorting
data=df_nse.sort_index(ascending=True,axis=0)
new_data=pd.DataFrame(index=range(0,len(df_nse)),columns=['Date','Close'])

In [204]:
for i in range(0,len(data)):
    new_data["Date"][i]=data['Date'][i]
    new_data["Close"][i]=data["Close"][i]

new_data.index=new_data.Date
new_data.drop("Date",axis=1,inplace=True)

dataset=new_data.values

In [205]:
data

,Date,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
Date,,,,,,,,
2013-10-08,2013-10-08,157.00,157.80,155.20,155.80,155.80,1720413.0,2688.94
2013-10-09,2013-10-09,155.70,158.20,154.15,155.30,155.55,2049580.0,3204.49
2013-10-10,2013-10-10,156.00,160.80,155.85,160.30,160.15,3124853.0,4978.80
2013-10-11,2013-10-11,161.15,163.45,159.00,159.80,160.05,1880046.0,3030.76
2013-10-14,2013-10-14,160.85,161.45,157.70,159.30,159.45,1281419.0,2039.09
...,...,...,...,...,...,...,...,...
2018-10-01,2018-10-01,234.55,234.60,221.05,230.30,230.90,1534749.0,3486.05
2018-10-03,2018-10-03,230.00,237.50,225.75,226.45,227.60,1708590.0,3960.27
2018-10-04,2018-10-04,223.50,227.80,216.15,217.25,218.20,1728786.0,3815.79


Dataset split to Train and Test

In [206]:
# 
train=dataset[0:987,:]
valid=dataset[987:,:]

Scaling of data

In [207]:

scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)

In [208]:
x_train, y_train = [],[]

for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])

In [209]:
x_train,y_train=np.array(x_train),np.array(y_train)
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

MODEL LOADING

In [210]:
model = load_model("data/saved_lstm_model.h5")

In [211]:
inputs=new_data[len(new_data)-len(valid)-60:].values
inputs=inputs.reshape(-1,1)
inputs=scaler.transform(inputs)

In [212]:
X_test=[]
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test=np.array(X_test)

X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
closing_price=model.predict(X_test)
closing_price=scaler.inverse_transform(closing_price)

8/8 [==============================] - 1s 8ms/step


DATASET SPLIT TO VALIDATION AND TRAINING

In [213]:
train=new_data[:987]
valid=new_data[987:]
# valid['Predictions']=closing_price

In [214]:
valid.loc[:,['Predictions']] = closing_price

In [215]:
valid.head()

,Close,Predictions
Date,,
2017-10-09,208.3,206.019699
2017-10-10,208.45,207.602585
2017-10-11,209.4,208.739929
2017-10-12,212.0,209.619034
2017-10-13,210.25,210.684631


REGREESION METRICS FOR MODEL

In [216]:
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

In [217]:
mae = mean_absolute_error(valid["Close"], valid["Predictions"])
print("mean_absolute_error",mae)


mse = mean_squared_error(valid["Close"], valid["Predictions"])
print("mean_squared_error",mse)


rmse = np.sqrt(mean_squared_error(valid["Close"], valid["Predictions"]))
print("root_mean_squared_error",rmse)

r2 = r2_score(valid["Close"], valid["Predictions"])
print("r-squared_score",r2)

mean_absolute_error 7.670436231551631
mean_squared_error 104.94695330478396
root_mean_squared_error 10.244362025269508
r-squared_score 0.8708098540636484


In [218]:
regression_metrics = {
    'MAE': mae,
    'MSE': mse,
    'RMSE': rmse,
    'R2': r2
}

# DASHBOARD

APP SERVER

In [219]:
app = dash.Dash(__name__)
server = app.server

APP LAYOUT

In [228]:
app.layout = html.Div([
   
    html.H1("Stock Price Analysis Dashboard"),
   
    dcc.Tabs(id="tabs", children=[
		# tab1
		dcc.Tab(label="Model and TATA Stock Analysis",className='custom-tab',children=[
            html.Div([
                html.H2("Actual closing price", className='custom-content'),
				dcc.Graph(
					id="Actual Data",
					figure={
						"data":[
							go.Scatter(
								x=train.index,
								y=train["Close"],
								mode='lines'
							)

						],
						"layout":go.Layout(
							title='Actual Closing Rate(2013-2017)',
							xaxis={'title':'Date'},
							yaxis={'title':'Closing Rate'}
						)
					}
				),
			]),
            
			# 2 panes 
			html.Div([
                # left pane
				html.Div([
            	    html.H2("Model Metrics Table", className='custom-content'),
					dash_table.DataTable(
        				id='metrics-table',
        				columns=[
        			    	{'name': 'Metric', 'id': 'Metric'},
        			    	{'name': 'Value', 'id': 'Value'}
        				],
        				data=[{'Metric': metric, 'Value': value} for metric, value in regression_metrics.items()],
                	),
				], className='pane pane1' ),
            
				# right pane
            	html.Div([
					html.H2("LSTM Predicted closing price", className='custom-content'),
					dcc.Graph(
						id="Predicted Data",
						figure={
							"data":[
								go.Scatter(
									x=valid.index,
									y=valid["Predictions"],
									mode='lines'
								)
							],
							"layout":go.Layout(
								title='Predicted Closing Rate(2017-2018)',
								xaxis={'title':'Date'},
								yaxis={'title':'Closing Rate'}
							)
						}
					),
				], className='pane pane2'),
            ],className='pane'),
			
		]),
        
        # dcc.Tab(label='NSE-TATAGLOBAL Stock Data', className='custom-tab', children=[
		# 	html.Div([
		# 		html.H2("Actual closing price", className='custom-content'),
		# 		dcc.Graph(
		# 			id="Actual Data",
		# 			figure={
		# 				"data":[
		# 					go.Scatter(
		# 						x=train.index,
		# 						y=train["Close"],
		# 						mode='lines'
		# 					)

		# 				],
		# 				"layout":go.Layout(
		# 					title='Actual Closing Rate(2013-2014)',
		# 					xaxis={'title':'Date'},
		# 					yaxis={'title':'Closing Rate'}
		# 				)
		# 			}
		# 		),
		# 		html.H2("LSTM Predicted closing price", className='custom-content'),
		# 		dcc.Graph(
		# 			id="Predicted Data",
		# 			figure={
		# 				"data":[
		# 					go.Scatter(
		# 						x=valid.index,
		# 						y=valid["Predictions"],
		# 						mode='lines'
		# 					)
		# 				],
		# 				"layout":go.Layout(
		# 					title='Predicted Closing Rate(2017-2018)',
		# 					xaxis={'title':'Date'},
		# 					yaxis={'title':'Closing Rate'}
		# 				)
		# 			}
		# 		),
        #         html.H2("Model Metrics Table", className='custom-content'),
        #         dash_table.DataTable(
        # 			id='metrics-table',
        # 			columns=[
        # 			    {'name': 'Metric', 'id': 'Metric'},
        # 			    {'name': 'Value', 'id': 'Value'}
        # 			],
        # 			data=[{'Metric': metric, 'Value': value} for metric, value in regression_metrics.items()]
        #         ),
		# 	])        		
        # ]),

		# tab2
        dcc.Tab(label='NSE-TATAGLOBAL Stock Analysis',className='custom-tab', children=[
            html.Div([
                html.H2("Tata Stock Actual vs Predicted closing price", className='custom-content'),

                dcc.Dropdown(id= 'my-dropdown3',
                             options=[{'label': 'Actual Closing', 'value': 'Close'},
                                      {'label': 'Predicted Closing', 'value': 'Predictions'}],
                            multi=True,value=['Close'], className='dropdown-menu'),
                dcc.Graph(id='closing'),
            ]),
            
			html.Div([
                html.H2("Other Stats", className='custom-content'),
                dcc.Checklist(
        			id='toggle-buttons',
        			options=[
            			{'label': 'Open', 'value': 'Open'},
            			{'label': 'High', 'value': 'High'},
            			{'label': 'Low', 'value': 'Low'},
            			{'label': 'Last', 'value': 'Last'},
        			],
        			value=['Open'],  # Default value is an empty list
                    className='dash-checklist'
    			),
                dcc.Graph(id='toggle-button-output'),			
			]),
            
			html.Div([
                html.H3("TATA GLOBAL stock Volume", className='custom-content'),
                dcc.Graph(
					id="Predicted Data",
					figure={
						"data":[
							go.Scatter(
								x=df_nse.index,
								y=df_nse["Total Trade Quantity"],
								mode='lines'
							)
						],
						"layout":go.Layout(
							title='Trade Quantity for Tata Global Stock over Time',
							xaxis={'title':'Date'},
							yaxis={'title':'Transaction Volume'}
						)
					}
				),
			])
        ]),

		# tab3
        dcc.Tab(label='Stock Analysis (AAPL, TSLA, etc)',className='custom-tab', children=[
            html.Div([
                html.H2("Stocks High vs Lows", className='custom-content'),
              
                dcc.Dropdown(id='my-dropdown',
                             options=[{'label': 'Tesla', 'value': 'TSLA'},
                                      {'label': 'Apple','value': 'AAPL'}, 
                                      {'label': 'Facebook', 'value': 'FB'}, 
                                      {'label': 'Microsoft','value': 'MSFT'}], 
                             multi=True,value=['FB'], className='dropdown-menu'),
                dcc.Graph(id='highlow'),


                html.H2("Stocks Market Volume", className='custom-content'),
                dcc.Dropdown(id='my-dropdown2',
                             options=[{'label': 'Tesla', 'value': 'TSLA'},
                                      {'label': 'Apple','value': 'AAPL'}, 
                                      {'label': 'Facebook', 'value': 'FB'},
                                      {'label': 'Microsoft','value': 'MSFT'}], 
                             multi=True,value=['FB'],className='dropdown-menu'),
                dcc.Graph(id='volume')
            ]),
        ])


    ])
])

APP CALLBACKS

In [221]:
@app.callback(Output('highlow', 'figure'),
              [Input('my-dropdown', 'value')])
def update_graph_highlow(selected_dropdown):
    dropdown = {"TSLA": "Tesla","AAPL": "Apple","FB": "Facebook","MSFT": "Microsoft",}
    trace1 = []
    trace2 = []
    for stock in selected_dropdown:
        trace1.append(
          go.Scatter(x=df[df["Stock"] == stock]["Date"],
                     y=df[df["Stock"] == stock]["High"],
                     mode='lines', opacity=0.7, 
                     name=f'High {dropdown[stock]}',textposition='bottom center'))
        trace2.append(
          go.Scatter(x=df[df["Stock"] == stock]["Date"],
                     y=df[df["Stock"] == stock]["Low"],
                     mode='lines', opacity=0.6,
                     name=f'Low {dropdown[stock]}',textposition='bottom center'))
    traces = [trace1, trace2]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data,
              'layout': go.Layout(colorway=["#5E0DAC", '#FF4F00', '#375CB1', 
                                            '#FF7400', '#FFF400', '#FF0056'],
            height=600,
            title=f"High and Low Prices for {', '.join(str(dropdown[i]) for i in selected_dropdown)} Over Time",
            xaxis={"title":"Date",
                   'rangeselector': {'buttons': list([{'count': 1, 'label': '1M', 
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'count': 6, 'label': '6M', 
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'step': 'all'}])},
                   'rangeslider': {'visible': True}, 'type': 'date'},
             yaxis={"title":"Price (USD)"})}
    return figure

In [222]:
@app.callback(Output('closing','figure'),
              [Input('my-dropdown3', 'value')])
def update_graph_closing(selected_dropdown_close):
    dropdown= {"Close": "Actual","Predictions": "Predicted"}
    trace1=[]
    for stock in selected_dropdown_close:
        trace1.append(
            go.Scatter(x=valid.index,
                       y=valid[stock],
                       mode = 'lines', opacity=0.7,
                       name=f'{dropdown[stock]} Closing ', textposition='bottom center'))
        
    traces = [trace1]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data, 
              'layout': go.Layout(colorway=["#5E0DAC", '#FF4F00', '#375CB1', 
                                            '#FF7400', '#FFF400', '#FF0056'],
            height=600,
            title=f"{''.join(str(dropdown[i]) for i in selected_dropdown_close)} Prices Over Time",
            xaxis={"title":"Date",
                   'rangeselector': {'buttons': list([{'count': 1, 'label': '1M', 
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'count': 6, 'label': '6M',
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'step': 'all'}])},
                   'rangeslider': {'visible': True}, 'type': 'date'},
             yaxis={"title":"Cosing Price"})}
    return figure

In [223]:
@app.callback(Output('volume', 'figure'),
              [Input('my-dropdown2', 'value')])
def update_graph_volume(selected_dropdown_value):
    dropdown = {"TSLA": "Tesla","AAPL": "Apple","FB": "Facebook","MSFT": "Microsoft",}
    trace1 = []
    for stock in selected_dropdown_value:
        trace1.append(
          go.Scatter(x=df[df["Stock"] == stock]["Date"],
                     y=df[df["Stock"] == stock]["Volume"],
                     mode='lines', opacity=0.7,
                     name=f'Volume {dropdown[stock]}', textposition='bottom center'))
    traces = [trace1]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data, 
              'layout': go.Layout(colorway=["#5E0DAC", '#FF4F00', '#375CB1', 
                                            '#FF7400', '#FFF400', '#FF0056'],
            height=600,
            title=f"Market Volume for {', '.join(str(dropdown[i]) for i in selected_dropdown_value)} Over Time",
            xaxis={"title":"Date",
                   'rangeselector': {'buttons': list([{'count': 1, 'label': '1M', 
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'count': 6, 'label': '6M',
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'step': 'all'}])},
                   'rangeslider': {'visible': True}, 'type': 'date'},
             yaxis={"title":"Transactions Volume"})}
    return figure

In [224]:
@app.callback(Output('toggle-button-output', 'figure'),
              [Input('toggle-buttons', 'value')])
def update_output(selected_values):
    dropdown = {"Open": "Open","High": "High","Low": "Low","Last": "Last",}
    trace1 = []
    for stock in selected_values:
        trace1.append(
          go.Scatter(x=df_nse.index,
                       y=df_nse[stock],
                       mode = 'lines', opacity=0.7,
                       name=f'{dropdown[stock]} ', textposition='bottom center'))
    traces = [trace1]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data, 
              'layout': go.Layout(colorway=["#5E0DAC", '#FF4F00', '#375CB1', 
                                            '#FF7400', '#FFF400', '#FF0056'],
            height=600,
            title=f"{', '.join(str(dropdown[i]) for i in selected_values)} values Over Time",
            xaxis={"title":"Date",
                   'rangeselector': {'buttons': list([{'count': 1, 'label': '1M', 
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'count': 6, 'label': '6M',
                                                       'step': 'month', 
                                                       'stepmode': 'backward'},
                                                      {'step': 'all'}])},
                   'rangeslider': {'visible': True}, 'type': 'date'},
             yaxis={"title":"Prices"})}
    return figure

APP SERVER START

In [225]:
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)